In [1]:
import json

In [2]:
gt_json = "/home/hqyone/mnt/2tb/github/cancer_rcnn/data/500_Json/1456.json"
pred_json = "/home/hqyone/mnt/2tb/github/cancer_rcnn/data/output/1456/1456.json"

In [3]:
a = [3,4,2]
a.sort()
print(a)

[2, 3, 4]


In [4]:
# Get bbox from the boudary points
def get_bbox(points):
    # bbox=[x1,y1,x2,y2]
    bbox=None
    ys=[]
    xs=[]
    for p in points:
        ys.append(p[0])
        xs.append(p[1])
    if len(ys)>=3 and len(xs)>=3:
        ys.sort()
        xs.sort()
        bbox=[xs[0],ys[0],xs[-1],ys[-1]]
    return(bbox)

def get_bbox_iou(bbox1, bbox2):
    iou_box=None
    if not (bbox1[0]>bbox2[2] or bbox1[2]<bbox2[0]) and not (bbox1[1]>bbox2[3] or bbox1[3]<bbox2[1]):
        iou_box=[max(bbox1[0],bbox2[0]),max(bbox1[1],bbox2[1]), min(bbox1[2],bbox2[2]), min(bbox1[3],bbox2[3]) ]
    return iou_box

def get_bbox_region(bbox):
    return(abs(bbox[2]-bbox[0])*(bbox[3]-bbox[1]))

# Test get_bbox
points_1=[[1,3],[2,1],[3,3]]
points_2=[[4,3],[2,1],[4,1],[5,2]]
bbox1=get_bbox(points_1)
bbox2=get_bbox(points_2)
print(bbox1)
print(bbox2)
# Test get_bbox_iou
bbox_iou = get_bbox_iou(bbox1, bbox2)
print(bbox_iou)
# Test get_bbox_region
get_bbox_region(bbox_iou)

[1, 1, 3, 3]
[1, 2, 3, 5]
[1, 2, 3, 3]


2

In [5]:
def get_closed_shape(target_shape, shapes):
    best_shape=None
    best_iou_ratio = 0
    for i, s in enumerate(shapes):
        bbox = get_bbox(target_shape['points'])
        pred_bbox = get_bbox(s['points'])
        if bbox and pred_bbox:
            bbox_iou = get_bbox_iou(bbox, pred_bbox)
            if bbox_iou:
                bbox_iou_region = get_bbox_region(bbox_iou)
                pre_bbox_region = get_bbox_region(pred_bbox)
                bbox_region = get_bbox_region(bbox)
                if (bbox_iou_region<0):
                    print(bbox)
                    print(bbox_iou)
                    print(pred_bbox)
                if bbox_region>0:
                    bbox_iou_ratio = bbox_iou_region/(bbox_region+pre_bbox_region-bbox_iou_region)
                    if bbox_iou_ratio>0.5:
                        if bbox_iou_ratio>best_iou_ratio:
                            best_iou_ratio=bbox_iou_ratio
                            best_shape = s
    return [best_shape, round(best_iou_ratio, 2)]

# Test get_closed_shape
points_1=[[1,3],[2,1],[3,3]]
points_2=[[4,3],[2,1],[4,1],[5,2]]
target_shape={"points":points_1}
shapes=[{"points":points_2}]
get_closed_shape(target_shape, shapes)
           

[None, 0]

In [6]:


def compare_json(gt_json, pred_json, confusion_matrix):
    """compare_json _summary_
    The function is used to compare ground truth json with predicted json files
    To got confusion matrix with multiple classes
    Arguments:
        gt_json {string} -- ground truth json file
        pred_json {string} -- predict json file
        class_labels{list} -- A list of all possible class names
    """
    with open(gt_json) as gtJSON:
        gt_data = json.load(gtJSON)
        gt_image_name = gt_data['imagePath']
        gt_image_height = gt_data['imageHeight']
        gt_image_width = gt_data['imageWidth']
        gt_shapes = gt_data["shapes"]

    with open(pred_json) as gtJSON:
        pred_data = json.load(gtJSON)
        pred_image_name = pred_data['imagePath']
        pred_image_height = pred_data['imageHeight']
        pred_image_width = pred_data['imageWidth']
        pred_shapes = pred_data["shapes"]

    label_index_dic={
        "yin":0,
        "yin-yang":1,
        "yang":2
    }

    for i, gs in enumerate(gt_shapes):
        [best_shape, best_iou_ratio] = get_closed_shape(gs, pred_shapes)
        gt_label=gs["label"]
        if best_shape:
            pred_label=best_shape["label"]
            if gt_label=="yin-yang":
                print(best_iou_ratio)
            confusion_matrix[gt_label][label_index_dic[pred_label]]+=1
        else:
            confusion_matrix[gt_label][3]+=1
    return confusion_matrix



In [7]:
# The result matrix
confusion_matrix = {
    "yin":[0,0,0,0],
    "yin-yang":[0,0,0,0],
    "yang":[0,0,0,0]
}

print(compare_json(gt_json, pred_json, confusion_matrix)) 

{'yin': [25, 0, 0, 48], 'yin-yang': [0, 0, 0, 0], 'yang': [0, 0, 0, 0]}


In [9]:
import glob
gt_json_dir="/home/hqyone/mnt/2tb/github/cancer_rcnn/data/test_dataset/test-50-image_json"
predict_dir = "/home/hqyone/mnt/2tb/github/cancer_rcnn/data/test_dataset/modelT1"
# predict_dir=gt_json_dir
# The result matrix
confusion_matrix = {
    "yin":[0,0,0,0],
    "yin-yang":[0,0,0,0],
    "yang":[0,0,0,0]
}

for gt_json in glob.glob(f"{gt_json_dir}/*.json", recursive=True):
    filename = os.path.basename(gt_json).replace(".json","")
    print(filename)
    predict_json = f"{predict_dir}/{filename}.json"
    #predict_json = gt_json
    if os.path.isfile(predict_json):
        print(filename)
        compare_json(gt_json, predict_json, confusion_matrix)
        print(confusion_matrix)
print(confusion_matrix)

1042
1042
{'yin': [57, 0, 2, 0], 'yin-yang': [0, 0, 0, 0], 'yang': [0, 0, 9, 0]}
1044
1044
{'yin': [116, 0, 2, 0], 'yin-yang': [0, 0, 0, 0], 'yang': [0, 0, 11, 0]}
1046
1046
0.92
{'yin': [158, 2, 2, 1], 'yin-yang': [0, 1, 0, 0], 'yang': [0, 1, 15, 0]}
1048
1048
{'yin': [216, 2, 4, 1], 'yin-yang': [0, 1, 0, 0], 'yang': [0, 1, 22, 0]}
1049
1049
0.86
0.91
0.97
{'yin': [266, 9, 6, 1], 'yin-yang': [0, 2, 2, 0], 'yang': [0, 1, 24, 0]}
1051
1051
0.86
0.92
0.9
0.9
0.9
0.92
{'yin': [329, 10, 6, 4], 'yin-yang': [0, 8, 2, 0], 'yang': [0, 2, 29, 0]}
1052
1052
0.84
0.9
0.9
{'yin': [382, 13, 6, 5], 'yin-yang': [0, 10, 3, 0], 'yang': [0, 2, 32, 2]}
1053
1053
0.97
{'yin': [442, 13, 6, 5], 'yin-yang': [0, 10, 4, 0], 'yang': [0, 2, 37, 2]}
1055
1055
{'yin': [506, 13, 6, 5], 'yin-yang': [0, 10, 4, 0], 'yang': [0, 2, 37, 2]}
1058
1058
1.0
0.99
0.98
0.95
0.98
1.0
{'yin': [555, 13, 6, 6], 'yin-yang': [0, 13, 7, 0], 'yang': [0, 2, 41, 2]}
1059
1059
{'yin': [615, 13, 6, 6], 'yin-yang': [0, 13, 7, 0], 'yang': 

In [ ]:
# combine the ying-yang and yang cells
# calculate the confusion matrix


